In [1]:
import tensorflow as tf
import pandas as pd

#df_4forums = pd.read_json('4ForumsResults.json') # This file is too large to go through git
df_conme = pd.read_excel('convinceMe.xlsx')
df_credb = pd.read_excel('createDebate.xlsx')


2023-05-07 19:50:04.106174: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# install transformers if not already installed
%pip install transformers

#import the BERT model and tokenizer
from transformers import BertModel, BertTokenizer

if 'bertModel' not in locals() or 'tokenizer' not in locals():
    #BERT base model
    bertModel = BertModel.from_pretrained('bert-base-uncased')
    
    #BERT Tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Note: you may need to restart the kernel to use updated packages.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
df_credb.stance.value_counts()

unknown                       3051
prefers strict gun control    3051
opposes strict gun control    3051
undecided                     3051
other                         3051
Name: stance, dtype: int64

In [4]:
df_4forums.stance.value_counts()

undecided                                                                      149210
other                                                                          148799
unknown                                                                        148387
evolution occurs via purely natural mechanisms                                  42309
evolution does not occur                                                        42309
evolution involves more than purely natural mechanisms (intelligent design)     42309
pro-choice                                                                      36668
pro-life                                                                        36668
opposes strict gun control                                                      35966
prefers strict gun control                                                      35966
gay marriage should be legal                                                    18895
gay marriage should be illegal                        

In [5]:
df_conme.stance.value_counts()

gay marriage should be legal                                                   622
pro-choice                                                                     583
atheist                                                                        551
evolution occurs via purely natural mechanisms                                 498
pro-life                                                                       494
theist                                                                         447
evolution involves more than purely natural mechanisms (intelligent design)    384
gay marriage should be illegal                                                 349
opposes death penalty                                                          322
marijuana use should be legal                                                  304
supports death penalty                                                         278
opposes strict gun control                                                     217
huma

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [7]:
class BertClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits